In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [3]:
checkpoint = "bert-base-uncased"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
dataset = load_dataset("mwarchalowski/names-data")

In [31]:
dataset = load_dataset("mwarchalowski/names-data", split="train[:10%]")

In [32]:
dataset = dataset.filter(lambda e: e["name"] is not None)

Filter:   0%|          | 0/468400 [00:00<?, ? examples/s]

In [33]:
def tokenize_function(examples):
    return tokenizer(examples["name"], truncation=True)
    

In [21]:
dataset

Dataset({
    features: ['name', 'label'],
    num_rows: 468400
})

In [34]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/468400 [00:00<?, ? examples/s]

In [35]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [41]:
samples = tokenized_datasets[:8]
samples.items()

dict_items([('name', ['Murphy Facility Solutions', 'Mansell Building Solutions', 'Johnny Boulos', 'Gurdeep Singh', 'Stefan Birrer', 'ECG Egineering Consulting Group', 'Topdeck Travel', 'Tamar Stiebel']), ('label', [0, 0, 1, 1, 1, 0, 0, 1]), ('input_ids', [[101, 7104, 4322, 7300, 102], [101, 16042, 5349, 2311, 7300, 102], [101, 5206, 8945, 18845, 2015, 102], [101, 19739, 25547, 13699, 5960, 102], [101, 8852, 12170, 14343, 2099, 102], [101, 14925, 2290, 1041, 11528, 20550, 10552, 2177, 102], [101, 2327, 26547, 3604, 102], [101, 17214, 2906, 2358, 2666, 8671, 102]]), ('token_type_ids', [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]), ('attention_mask', [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]])])

In [43]:
samples = {k: v for k, v in samples.items() if k not in ["name", "label"]}

In [52]:
batch = data_collator(samples)
batch

{'input_ids': tensor([[  101,  7104,  4322,  7300,   102,     0,     0,     0,     0],
        [  101, 16042,  5349,  2311,  7300,   102,     0,     0,     0],
        [  101,  5206,  8945, 18845,  2015,   102,     0,     0,     0],
        [  101, 19739, 25547, 13699,  5960,   102,     0,     0,     0],
        [  101,  8852, 12170, 14343,  2099,   102,     0,     0,     0],
        [  101, 14925,  2290,  1041, 11528, 20550, 10552,  2177,   102],
        [  101,  2327, 26547,  3604,   102,     0,     0,     0,     0],
        [  101, 17214,  2906,  2358,  2666,  8671,   102,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0,

In [51]:
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 9]),
 'token_type_ids': torch.Size([8, 9]),
 'attention_mask': torch.Size([8, 9])}